In [1]:
import sys
import gym
import numpy as np
from collections import defaultdict
import pdb
import matplotlib.pyplot as plt
from Environment import *

In [9]:
Map  = np.array([['s','f','f','f','f','f','f'],
                 ['f','f','f','f','f','f','f'],
                 ['f','f','f','f','f','f','f'],
                 ['f','f','f','f','f','f','f'],
                 ['f','f','f','f','f','f','g'],
                 ['f','f','f','f','f','f','f'],
                 ['f','f','f','f','f','f','f']])

env = Environment(Map,False)

In [10]:
Q_values = defaultdict(lambda: np.random.randint(0,4,4).astype(np.float32))

In [11]:
action_to_num = {
    'u':0,
    'd':1,
    'r':2,
    'l':3
}

num_to_action = {
    '0':'u',
    '1':'d',
    '2':'r',
    '3':'l'
}

In [12]:
def sarsa(env,Q_values,num_iter,num_episodes,alpha, gamma=0.9):
    
    for it in range(num_iter):
        state = env.reset()
        action = num_to_action[str(np.argmax(Q_values[state]))]
        for eps in range(num_episodes):
            if np.random.rand() < 0.85:
                new_action = num_to_action[str(np.argmax(Q_values[state]))]
                new_state,reward,flag = env.step(action)
            else:
                new_action = num_to_action[str(np.random.randint(0,4))]
                new_state,reward,flag = env.step(action)
                
            Q_values[(state[0],state[1])][action_to_num[action]] += alpha*(reward+gamma*Q_values[(env.current_position[0],env.current_position[1])][action_to_num[new_action]]-Q_values[(state[0],state[1])][action_to_num[action]])
            state = new_state
            action = new_action
            if env.getState() == 'g':
                break
            
            
    return Q_values

In [28]:
Q_star = sarsa(env,Q_values,50000,60,0.9,0.9)

In [29]:
def QvalueToPolicy(Q,Map):
    policy = np.empty(Map.shape,dtype=object)
    
    for i in range(Map.shape[0]):
        for j in range(Map.shape[1]):
            policy[i,j] = num_to_action[str(np.argmax(Q[(i,j)]))]
            
    return policy

In [30]:
policy = QvalueToPolicy(Q_star,Map)
policy

array([['r', 'r', 'r', 'd', 'd', 'd', 'd'],
       ['l', 'u', 'd', 'r', 'r', 'd', 'd'],
       ['l', 'r', 'r', 'r', 'u', 'd', 'd'],
       ['d', 'd', 'r', 'l', 'd', 'd', 'r'],
       ['r', 'r', 'r', 'r', 'r', 'r', 'd'],
       ['u', 'u', 'u', 'r', 'r', 'u', 'u'],
       ['u', 'd', 'r', 'r', 'r', 'u', 'r']], dtype=object)